In [ ]:
%matplotlib inline

## Iris introduction course
# 5. Data Processing

**Learning Outcome**: by the end of this section, you will be able to explain the capabilities and functionality of Iris Cubes and Coordinates.

**Duration:** 1.5 hour

**Overview:**<br>
5.1 [Cube Arithmetic](#arithmetic)<br>
5.2 [Aggregation and Statistics](#agg_and_stats)<br>
5.3 [Exercise : statistics and visualisation](#ex_5)<br>
5.4 [Summary of the Section](#summary)

## Setup

In [ ]:
import iris
import numpy as np

## 5.1 Cube Arithmetic<a id='arithmetic'></a>

Basic mathematical operators exist on the cube to allow one to add, subtract, divide, multiply and perform other mathematical operations on cubes of a similar shape to one another:

In [ ]:
a1b = iris.load_cube(iris.sample_data_path('A1B_north_america.nc'))
e1 = iris.load_cube(iris.sample_data_path('E1_north_america.nc'))

print(e1.summary(True))
print(a1b)

In [ ]:
scenario_difference = a1b - e1
print(scenario_difference)

Notice that the resultant cube's name is now `unknown` and that resultant cube's `attributes` and `cell_methods` have disappeared; this is because these all differed between the two input cubes.

It is also possible to operate on cubes with numeric scalars, NumPy arrays and even cube coordinates:

In [ ]:
print(e1 * e1.coord('latitude'))

Cube broadcasting is also taking place, meaning that the two inputs (cube, coordinate, array, or even constant value) don't need to have the same shape:

In [ ]:
print(e1 + 5.0)

As we've just seen, we have the ability to update the cube's data directly. Whenever we do this though, we should be mindful of updating appropriate metadata on the cube:

In [ ]:
e1_hot = e1.copy()

e1_hot.data = np.ma.masked_less_equal(e1_hot.data, 280)
e1_hot.rename('air temperatures greater than 280K')
print(e1_hot)

## 5.2 Cube aggregation and statistics<a id='agg_and_stats'></a>

Many standard univariate aggregations exist in Iris. Aggregations allow one or more dimensions of a cube to be statistically collapsed for the purposes of statistical analysis of the cube's data. Iris uses the term 'aggregators' to refer to the statistical operations that can be used for aggregation.

A list of aggregators is available at http://scitools.org.uk/iris/docs/latest/iris/iris/analysis.html.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cube = iris.load_cube(fname, 'air_potential_temperature')
print(cube.summary(True))

To take the vertical mean of this cube:

In [ ]:
print(cube.collapsed('model_level_number', iris.analysis.MEAN))

----

Some other types of statistical calculation are also provided.  See the following documentation areas :

 * [Cube.collapsed](https://scitools.org.uk/iris/docs/latest/iris/iris/cube.html?highlight=collapsed#iris.cube.Cube.collapsed), as discussed above.
 * [Cube.rolling_window](https://scitools.org.uk/iris/docs/latest/iris/iris/cube.html?highlight=rolling#iris.cube.Cube.rolling_window).
 * [Cube.aggregated_by](https://scitools.org.uk/iris/docs/latest/iris/iris/cube.html?highlight=aggregated_by#iris.cube.Cube.aggregated_by), used with the [coord_categorisation module](https://scitools.org.uk/iris/docs/latest/iris/iris/coord_categorisation.html?highlight=categor#module-iris.coord_categorisation).  
This provides "group-by-and-reduce" type calculations -- these are explained later in section 6, "Advanced Concepts".

## 5.3 : Exercise 5 - statistics and visualisation<a id='ex_5'></a>

Let's apply all that we've learned about data processing and visualisation in Iris. We will perform data processing and visualisation to compare two possible climate futures scenarios, called the A1B scenario and the E1 scenario.

1\. Load as cubes the datasets found at `iris.sample_data_path('E1_north_america.nc')` and `iris.sample_data_path('A1B_north_america.nc')`. Print the summary of each cube.

In [ ]:
cube_e1 = iris.load_cube(iris.sample_data_path('E1_north_america.nc'))
cube_a1b = iris.load_cube(iris.sample_data_path('A1B_north_america.nc'))
print(cube_e1.summary())
print(cube_a1b.summary())

2a\. Plot the following data in a single figure with three maps in one row:

 * the air temperature in the E1 scenario in 2099, 
 * the air temperature in the A1B scenario in 2099, and
 * the difference between the two scenarios.

Think about the most appropriate matplotlib colormap(s) to use for each plot. Hint: the different matplotlib colormaps can be seen at https://matplotlib.org/1.5.3/examples/color/colormaps_reference.html. 

In [ ]:
# First extract only one year from each.
def date_in_year_2099(time_cell):
    return time_cell.point.year == 2099

year_2099_constraint = iris.Constraint(time=date_in_year_2099)
e1_2099 = cube_e1.extract(year_2099_constraint)
a1b_2099 = cube_a1b.extract(year_2099_constraint)

# Print the results : only one timepoint, so they are 2-D.
print(e1_2099.summary(shorten=True))
print(a1b_2099.summary(shorten=True))

# Calculate the difference.
difference = a1b_2099 - e1_2099

# Plot them ...
import matplotlib.pyplot as plt
import iris.quickplot as qplt

plt.figure(figsize=(16,6))

originals_min = min(e1_2099.data.min(), a1b_2099.data.min())
originals_max = max(e1_2099.data.max(), a1b_2099.data.max())

# Display forecast temperatures on an absolute colour scale, with the same value range for both.
# The difference is rather hard to see, so specify a high-contrast colormap to help with that.
plt.subplot(1, 3, 1)
qplt.pcolormesh(e1_2099, vmin=originals_min, vmax=originals_max, cmap='brg')
plt.title('Scenario E1, year 2099')

plt.subplot(1, 3, 2)
qplt.pcolormesh(a1b_2099, vmin=originals_min, vmax=originals_max, cmap='brg')
plt.title('Scenario A1B, year 2099')

plt.subplot(1, 3, 3)
# Calculate symmetrical limits for a zero-centred value range.
diff_min, diff_max = np.min(difference.data), np.max(difference.data)
diff_maxscale = max(diff_max, -diff_min)
# Plot with a 'diverging' colormap, suitable to the zero-centred value range.
qplt.pcolormesh(difference, vmin=-diff_maxscale, vmax=diff_maxscale, cmap='bwr')
plt.title('Temperature difference A1B - E1,\n year 2099')

print('Note: the last plot is the difference.\n'
      'This uses a "diverging" colour scale to show values above and below zero.\n',
      'All the values are positive, so it actually only has shades of red.')


2b\. What information do your plots show? 

3\. Produce cubes that describe the area-averaged air temperature over time for each scenario. Calculate the model difference between these two cubes.

HINT: see the documentation on [iris.cube.Cube.collapsed](https://scitools.org.uk/iris/docs/latest/iris/iris/cube.html#iris.cube.Cube.collapsed)
and [iris.analysis.cartography.area_weights](https://scitools.org.uk/iris/docs/latest/iris/iris/analysis/cartography.html#iris.analysis.cartography.area_weights)


In [ ]:
import iris.analysis.cartography as icart
for cube in (cube_e1, cube_a1b):
    for axis in ('x', 'y'):
        coord = cube.coord(axis=axis)
        if not coord.has_bounds():
            coord.guess_bounds()

area_weights = icart.area_weights(cube_e1)

e1_areamean = cube_e1.collapsed(['latitude', 'longitude'], iris.analysis.MEAN, weights=area_weights)
a1b_areamean = cube_a1b.collapsed(['latitude', 'longitude'], iris.analysis.MEAN, weights=area_weights)
difference_areamean = a1b_areamean - e1_areamean
difference_areamean.rename('Temperature difference, A1B - E1')
print(difference_areamean)

4\. Produce a single line plot with the data from the two cubes you produced in part 3. Make sure you label the lines you plot.

In [ ]:
# Use iplt rather than qplt, to avoid overlapping titles.
import iris.plot as iplt
iplt.plot(e1_areamean, '+-', color='blue', label='E1')
iplt.plot(a1b_areamean, '+-', color='red', label='A1B')
ax = plt.gca()
plt.legend()
plt.show()

In [ ]:
#
# Extended version 
#  - plot the difference also  
#  - use two different Y axes
#  - control Y limits to reduce overlap
#

# Use iplt rather than qplt, to avoid overlapping titles.
import iris.plot as iplt
iplt.plot(e1_areamean, '+', color='blue', label='E1 Temperature (K)')
iplt.plot(a1b_areamean, '+', color='red', label='A1B Temperature (K)')
ax = plt.gca()
# Modify Y limits slightly to avoid overlaps
y0, y1 = ax.get_ylim()
margin = 0.2 * (y1 - y0)
y0 = y0 - margin
ax.set_ylim(y0, y1)
ax.set_ylabel('Temperature (K)')

# Use twinx to make a second axes with a shared X axis.
ax2 = ax.twinx()
# Plot the difference signal on that.
iplt.plot(difference_areamean, '.-', color='black', label='A1B-E1 (difference)', axes=ax2)
# Add a zero-line
x0, x1 = ax2.get_xlim()
plt.hlines(0.0, x0, x1, color='grey', linewidth=0.5, linestyle='--')
ax2.set_ylabel('difference (K)')
# Modify Y limits slightly to avoid overlaps
y0, y1 = ax2.get_ylim()
margin = 0.2 * (y1 - y0)
y1 = y1 + margin
ax2.set_ylim(y0, y1)

# Matplotlib seems to have a problem with building a combined legend. This fixes it.
handles1, labels1 = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
plt.legend(handles1 + handles2, labels1 + labels2)

plt.show()

5\. Extract and compare the area-averaged air temperature values for both scenarios in 1980 and 2099. What conclusions can you draw from the values you've extracted?

----

## 5.4 Summary of Section: Data processing<a id='summary'></a>

In this section we learnt:
* cubes can be combined with arithmetic operators like addition, as for numpy arrays.  Broadcasting also works.
* coordinates can also be used in cube arithmetic. 
* operators are provided to perform statistical aggregations of cube data.
* statistics can be calculated over selected dimensions, identified by coordinates.
